<a href="https://colab.research.google.com/github/DevashishX/Deep-Learning-Models/blob/master/DLAZ_AEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoEncoders from Deep Learning A-Z on Udemy

We use autoencoders to predict the score (or stars) between 1 to 5 to know how much a user likes a movie based on his previous movie ratings

# Getting Dataset

In [1]:
!wget https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-AutoEncoders.zip
!unzip -q P16-AutoEncoders.zip
!mv AutoEncoders/* .
!unzip -q ml-100k.zip 
!unzip -q ml-1m.zip

--2020-05-09 06:25:27--  https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-AutoEncoders.zip
Resolving sds-platform-private.s3-us-east-2.amazonaws.com (sds-platform-private.s3-us-east-2.amazonaws.com)... 52.219.104.56
Connecting to sds-platform-private.s3-us-east-2.amazonaws.com (sds-platform-private.s3-us-east-2.amazonaws.com)|52.219.104.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17069342 (16M) [application/zip]
Saving to: ‘P16-AutoEncoders.zip’

P16-AutoEncoders.zi 100%[===================>]  16.28M  19.6MB/s    in 0.8s    

2020-05-09 06:25:28 (19.6 MB/s) - ‘P16-AutoEncoders.zip’ saved [17069342/17069342]



In [0]:
import numpy as np
import pandas as pd

In [0]:
# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Preparing the training set and the test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [0]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting Data into PyTorch Tensors

In [0]:
import torch

In [0]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Building the AutoEncoder

In [0]:
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

# Training AutoEncoder

In [0]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [13]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.7715)
epoch: 2 loss: tensor(1.0968)
epoch: 3 loss: tensor(1.0534)
epoch: 4 loss: tensor(1.0384)
epoch: 5 loss: tensor(1.0306)
epoch: 6 loss: tensor(1.0267)
epoch: 7 loss: tensor(1.0237)
epoch: 8 loss: tensor(1.0221)
epoch: 9 loss: tensor(1.0207)
epoch: 10 loss: tensor(1.0196)
epoch: 11 loss: tensor(1.0186)
epoch: 12 loss: tensor(1.0185)
epoch: 13 loss: tensor(1.0178)
epoch: 14 loss: tensor(1.0175)
epoch: 15 loss: tensor(1.0172)
epoch: 16 loss: tensor(1.0169)
epoch: 17 loss: tensor(1.0167)
epoch: 18 loss: tensor(1.0166)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0160)
epoch: 21 loss: tensor(1.0161)
epoch: 22 loss: tensor(1.0159)
epoch: 23 loss: tensor(1.0156)
epoch: 24 loss: tensor(1.0159)
epoch: 25 loss: tensor(1.0154)
epoch: 26 loss: tensor(1.0156)
epoch: 27 loss: tensor(1.0153)
epoch: 28 loss: tensor(1.0152)
epoch: 29 loss: tensor(1.0125)
epoch: 30 loss: tensor(1.0116)
epoch: 31 loss: tensor(1.0095)
epoch: 32 loss: tensor(1.0089)
epoch: 33 loss: t

# Testing AutoEncoder

In [16]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[(target == 0).unsqueeze(0)] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9546)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
